<a href="https://colab.research.google.com/github/jhonnybenna/machine_learning/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning and Data Mining Project: Spaceship Titanic
In this Colab Notebook we will use Machine Learning techniques to solve the Kaggle competions "Spaceship Titanic".



#Importing Dependencies


In [ ]:
URL = "https://raw.githubusercontent.com/jhonnybenna/machine_learning/main/"
OUTPUT_PATH = "kaggle_submissions/"
RANDOM_STATE = 42
TRAIN_SIZE = 0.8

DROPCOLUMNS=["PassengerId","Name","Transported"]

#Insert here the description of your test in order to submit to Kaggle
Description="Submission for the kaggle competition Spaceship Titanic "

In [ ]:
import os
import graphviz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    KFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from typing import List

import matplotlib as mpl
import matplotlib.pyplot as plt


In [ ]:
# For plotting purposes

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

#Importing Dataset from GitHub

In [ ]:
train = pd.read_csv(URL + "train.csv")
test = pd.read_csv(URL + "test.csv")

In [ ]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


#Data Inspection

In [ ]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

As we can see, the dataset contains both numerical data and non-numerical data (object).

Our goal is to predict the value of `Transported` based on the other features.

But, before training the model, let's make sure all of the data is in a proper format.

Since the `Cabin` columns contain data written in the format: `Deck`/`Cabin_num`/`Side`; it is necessary to split it into three different columns, each containing only one of the three variables

In [ ]:
train[["Deck", "Cabin_num", "Side"]] = train["Cabin"].str.split("/", expand=True)
# Dropping the column Cabin after splitting it into three separate columns
train.drop('Cabin', axis='columns', inplace=True)

In [ ]:
test[["Deck", "Cabin_num", "Side"]] = test["Cabin"].str.split("/", expand=True)
# Dropping the column Cabin after splitting it into three separate columns
test.drop('Cabin', axis='columns', inplace=True)

In [ ]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [ ]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Cabin_num,Side
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S


## Ordinal Encoding

Before using Ordinal Encoding:

In [ ]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Deck             object
Cabin_num        object
Side             object
dtype: object

In [ ]:
ord_enc = OrdinalEncoder()
train_code = train.copy()

for col in train_code.select_dtypes(include=['object', 'bool']).columns:
    train_code[col] = ord_enc.fit_transform(train_code[[col]])

train_code.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0.0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,5252.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,4502.0,1.0,5.0,0.0,1.0
2,2.0,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,457.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,7149.0,0.0,0.0,0.0,1.0
4,4.0,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,8319.0,1.0,5.0,1.0,1.0


After using Ordinal Encoding:

In [ ]:
train_code.dtypes

PassengerId     float64
HomePlanet      float64
CryoSleep       float64
Destination     float64
Age             float64
VIP             float64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name            float64
Transported     float64
Deck            float64
Cabin_num       float64
Side            float64
dtype: object

##Dealing with Missing Values


In [ ]:
# counts not_NA values of a DataFrame
def count_missing(df: pd.DataFrame):
    number_of_missing_in_cols = df.shape[0] - df.count()
    return number_of_missing_in_cols.sort_values()

In [ ]:
count_missing(train)

PassengerId       0
Transported       0
Age             179
RoomService     181
Destination     182
FoodCourt       183
Spa             183
VRDeck          188
Deck            199
Cabin_num       199
Side            199
Name            200
HomePlanet      201
VIP             203
ShoppingMall    208
CryoSleep       217
dtype: int64

As we can see from the previous line of code, there are many missing values in each column except, obviously, the `Transported` (target) and `PassengerID` ones.

This does not constitute a problem, as tensorflow is able to deal with missing values.

In [ ]:
# Count the number of unique values in dataframe
cols_unique_vals_count = train.drop(columns=DROPCOLUMNS).nunique().sort_values()
cols_unique_vals_count

CryoSleep          2
VIP                2
Side               2
HomePlanet         3
Destination        3
Deck               8
Age               80
ShoppingMall    1115
RoomService     1273
VRDeck          1306
Spa             1327
FoodCourt       1507
Cabin_num       1817
dtype: int64

In [ ]:
# If the column has only 2 unique values it is a binary col
BINARY_COLS = [col for col, val in cols_unique_vals_count.items() if val == 2]
BINARY_COLS

['CryoSleep', 'VIP', 'Side']

N.B: The `Transported` column has been dropped even though it's a binary column, as we assume it isn't missing any value; therefore we do not need to deal with it.

In [ ]:
# Maximum number of unique values which represent a Nominal (categorical) feature
NOMINAL_NUNIQUE_THRESHOLD = 10
NOMINAL_COLS = [
    col
    for col, val in cols_unique_vals_count.items()
    if val > 2 and val < NOMINAL_NUNIQUE_THRESHOLD
]
NOMINAL_COLS

['HomePlanet', 'Destination', 'Deck']

In [ ]:
# Maximum number of unique values which represent a Numerical (continuous) feature
NUMERICAL_COLS = [
    col
    for col, val in cols_unique_vals_count.items()
    if val > NOMINAL_NUNIQUE_THRESHOLD
]
NUMERICAL_COLS

['Age',
 'ShoppingMall',
 'RoomService',
 'VRDeck',
 'Spa',
 'FoodCourt',
 'Cabin_num']

In [ ]:
COLS = list(np.concatenate((BINARY_COLS, NOMINAL_COLS, NUMERICAL_COLS)))
COLS

['CryoSleep',
 'VIP',
 'Side',
 'HomePlanet',
 'Destination',
 'Deck',
 'Age',
 'ShoppingMall',
 'RoomService',
 'VRDeck',
 'Spa',
 'FoodCourt',
 'Cabin_num']

## Impute Missing Values

### Simple Imputation

In [ ]:
def array_to_dataframe(arr: np.ndarray, columns: List[str]):
    return pd.DataFrame(data=arr, columns=columns)

#### Constant

In [ ]:
constant_imp_arr = SimpleImputer(strategy="constant").fit_transform(train_code)
constant_imp_df = array_to_dataframe(arr=constant_imp_arr, columns=train_code.columns)
constant_imp_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0.0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,5252.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,4502.0,1.0,5.0,0.0,1.0
2,2.0,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,457.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,7149.0,0.0,0.0,0.0,1.0
4,4.0,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,8319.0,1.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688.0,1.0,0.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,3524.0,0.0,0.0,1795.0,0.0
8689,8689.0,0.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4780.0,0.0,6.0,548.0,1.0
8690,8690.0,0.0,0.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,3002.0,1.0,6.0,551.0,1.0
8691,8691.0,1.0,0.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1596.0,0.0,4.0,1385.0,1.0


In [ ]:
count_missing(constant_imp_df)

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
Deck            0
Cabin_num       0
Side            0
dtype: int64

#### Median

In [ ]:
median_imp_arr = SimpleImputer(strategy="median").fit_transform(train_code)
median_imp_df = array_to_dataframe(arr=median_imp_arr, columns=train_code.columns)
median_imp_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0.0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,5252.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,4502.0,1.0,5.0,0.0,1.0
2,2.0,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,457.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,7149.0,0.0,0.0,0.0,1.0
4,4.0,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,8319.0,1.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688.0,1.0,0.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,3524.0,0.0,0.0,1795.0,0.0
8689,8689.0,0.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4780.0,0.0,6.0,548.0,1.0
8690,8690.0,0.0,0.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,3002.0,1.0,6.0,551.0,1.0
8691,8691.0,1.0,0.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1596.0,0.0,4.0,1385.0,1.0


In [ ]:
count_missing(median_imp_df)

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
Deck            0
Cabin_num       0
Side            0
dtype: int64

### kNN Imputation

In [ ]:
knn_imp_arr = KNNImputer().fit_transform(train_code)
knn_imp_df = array_to_dataframe(arr=knn_imp_arr, columns=train_code.columns)
knn_imp_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0.0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,5252.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,4502.0,1.0,5.0,0.0,1.0
2,2.0,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,457.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,7149.0,0.0,0.0,0.0,1.0
4,4.0,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,8319.0,1.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688.0,1.0,0.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,3524.0,0.0,0.0,1795.0,0.0
8689,8689.0,0.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4780.0,0.0,6.0,548.0,1.0
8690,8690.0,0.0,0.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,3002.0,1.0,6.0,551.0,1.0
8691,8691.0,1.0,0.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1596.0,0.0,4.0,1385.0,1.0


In [ ]:
count_missing(knn_imp_df)

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
Deck            0
Cabin_num       0
Side            0
dtype: int64

### Model based Imputation

#### IterativeImputer

In [ ]:
iter_imp_arr = IterativeImputer().fit_transform(train_code)
iter_imp_df = array_to_dataframe(arr=iter_imp_arr, columns=train_code.columns)
iter_imp_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Cabin_num,Side
0,0.0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,5252.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,4502.0,1.0,5.0,0.0,1.0
2,2.0,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,457.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,7149.0,0.0,0.0,0.0,1.0
4,4.0,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,8319.0,1.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688.0,1.0,0.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,3524.0,0.0,0.0,1795.0,0.0
8689,8689.0,0.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4780.0,0.0,6.0,548.0,1.0
8690,8690.0,0.0,0.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,3002.0,1.0,6.0,551.0,1.0
8691,8691.0,1.0,0.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1596.0,0.0,4.0,1385.0,1.0


In [ ]:
count_missing(iter_imp_df)

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
Deck            0
Cabin_num       0
Side            0
dtype: int64

# Model Training

In [ ]:
def cmatrix(y_true: pd.DataFrame, y_pred: pd.DataFrame):
    data = confusion_matrix(y_true, y_pred)
    cmatrix_df = pd.DataFrame(
        data, columns=np.unique(y_pred), index=np.unique(y_true)
    )
    cmatrix_df.index.name = 'Actual'
    cmatrix_df.columns.name = 'Predicted'

    plt.figure(figsize=(6,5))

    # set label size
    sns.set(font_scale=1.2)
    # set font size
    axs = sns.heatmap(
        cmatrix_df, cmap="Blues", annot=True, fmt='', annot_kws={"size": 16}
    )
    axs.xaxis.tick_bottom()
    axs.yaxis.tick_left()

def transported_cm(y_true: pd.DataFrame, y_pred: pd.DataFrame):
    cmatrix(y_true, y_pred)
    plt.title("Transported passengers\n")

In [ ]:
ts_df = iter_imp_df

X = ts_df.drop(DROPCOLUMNS, axis='columns')
y = ts_df[["Transported"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=RANDOM_STATE
)

## Random Forest

### Hyperparameter Tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=300, stop=500, num=20)]
# Supported criteria in random forest
criterion = ['gini', 'entropy', 'log_loss']
# Number of features to consider at every split
max_features = ['sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Random grid
random_grid = {
    'n_estimators' : n_estimators,
    'criterion' : criterion,
    'max_features' : max_features,
    'max_depth' : max_depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'bootstrap' : bootstrap
}

rnd_clf = RandomForestClassifier()
rnd_random = RandomizedSearchCV(
    estimator = rnd_clf,
    param_distributions = random_grid,
    n_iter = 50,
    cv = StratifiedKFold(n_splits=5),
    scoring = 'accuracy'
)

In [ ]:
# Random grid
random_grid = {
    'n_estimators' : np.linspace(start=300, stop=500, num=20),
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_features' : ['sqrt', 'log2'],
    'max_depth' : np.linspace(10, 100, num=10),
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    'ccp_alpha' : np.arange(0.01, 0.055, step=0.005),
    'bootstrap' : [True, False]
}

rnd_clf = RandomForestClassifier()
rnd_random = RandomizedSearchCV(
    estimator = rnd_clf,
    param_distributions = random_grid,
    n_iter = 50,
    cv = StratifiedKFold(n_splits=5),
    scoring = 'accuracy'
)

In [ ]:
rnd_random.fit(X_train, y_train.to_numpy().ravel())

ValueError: ignored

In [ ]:
rnd_random.best_score_

In [ ]:
rnd_random.best_params_

In [ ]:
# Grid space
grid_space = {
    'n_estimators' : np.arange(350, 450, step=10),
    'criterion' : criterion,
    'max_features' : ['log2'],
    'max_depth' : np.arange(40, 60, step=1),
    'min_samples_split' : min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'bootstrap' : bootstrap
}

rnd_clf = RandomForestClassifier()

# Grid Search
grid_search = GridSearchCV(
    estimator = rnd_clf,
    param_grid = grid_space,
    cv = StratifiedKFold(n_splits=5),
    scoring = 'accuracy'
)

In [ ]:
grid_search.fit(X_train, y_train.to_numpy().ravel())

In [ ]:
rnd_clf = RandomForestClassifier(
   n_estimators = 468,
   min_samples_split = 5,
   min_samples_leaf = 2,
   max_features = 'log2',
   max_depth = 20,
   criterion = 'entropy',
   bootstrap = False,
   random_state = RANDOM_STATE
)

In [ ]:
rnd_clf.fit(X_train, y_train.to_numpy().ravel())
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
ft_imp = zip(X_train.columns.values, rnd_clf.feature_importances_)
ft_imp_sorted = sorted(ft_imp, key = lambda ft_imp: ft_imp[1], reverse = True)

for name, score in ft_imp_sorted:
    print(f"{name:15s} {score:.2f}")

In [ ]:
# plt.title('Feature Importance')
# plt.bar(range(X_train.shape[1]), importances[sorted_indices], align='center')
# plt.xticks(range(X_train.shape[1]), X_train.columns[sorted_indices], rotation=90)
# plt.tight_layout()
# plt.show()

In [ ]:
y_test_inv = np.rint(y_test).astype(int).to_numpy().ravel()

y_test_inv = np.where(y_test_inv == 1, True, False)
y_pred_inv = np.where(y_pred == 1, True, False)

transported_cm(y_test_inv, y_pred_inv)

# SVM


In [ ]:
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    # See https://medium.com/geekculture/svm-classification-with-sklearn-svm-svc-how-to-plot-a-decision-boundary-with-margins-in-2d-space-7232cb3962c0
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w0*x0 + w1*x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]
    m_hat = 1/w[1]
    gutter_up = decision_boundary + m_hat
    gutter_down = decision_boundary -  m_hat

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)


## Linear SVM

In [ ]:
from sklearn import metrics

def test_svm (model):

  y_pred = model.predict(X_test)
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
svm_clf = make_pipeline(StandardScaler(), LinearSVC(C=1, random_state=42))
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)

In [ ]:

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8041129313349599


##Non Linear SVM

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3),
    StandardScaler(),
    LinearSVC(C=10, max_iter=10000, random_state=42)
)

test_svm(poly_svm_clf)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, max_iter=10000, random_state=42))])

In [ ]:
y_pred = poly_svm_clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8041129313349599


In [ ]:
from sklearn.svm import SVC

kernel_svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel="poly", degree=3, coef0=1, C=5)
)

In [ ]:

kernel_svm_clf.fit(X_train, y_train)

test_svm(kernel_svm_clf)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.801673056814221
